In [1]:
import os
import pickle
import numpy as np

In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [3]:
embeddings_index = {}
with open( 'glove.6B.100d.txt', 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [4]:
with open('nodeMeta.pkl', 'rb') as f:
    nodeMeta = pickle.load(f)

In [5]:
corpus = []
for i in nodeMeta:
    corpus.append(nodeMeta[i]["abstract"])
print('Found %s texts.' % len(corpus))

Found 17500 texts.


In [6]:
corpus[3][:10]

['perform',
 'sewing',
 'two',
 'dual',
 'ramond',
 'reggeon',
 'vertices',
 'derive',
 'algorithm',
 'means']

In [7]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
t = Tokenizer()
t.fit_on_texts(corpus)
print (len(t.word_counts), 'words')

19801 words


In [8]:
vocab2idx = dict((i, t.word_index[i]) for i in t.word_index)
idx2vocab = dict((t.word_index[i], i) for i in t.word_index)
idx2vocab[0] = "<pad>"

In [9]:
# sequence length analysis
seq_length = [len(i) for i in corpus]
print(np.mean(seq_length), np.median(seq_length), np.min(seq_length), np.max(seq_length))

60.5432 55.0 2 237


In [10]:
from keras.preprocessing.sequence import pad_sequences
data = []
for i in corpus:
    seq = [t.word_index.get(j, 0) for j in i]
    pad_seq = pad_sequences([seq], maxlen=35, dtype='int32', padding='post', truncating='post', value=0.0)
    data.append(pad_seq)

In [11]:
cnt = 0
vocab_size = len(t.word_index)
embedding_matrix = np.zeros((vocab_size+1, 100))
for i in range(vocab_size+1):
    if idx2vocab[i] in embeddings_index:
        embedding_matrix[i] = embeddings_index[idx2vocab[i]]
    else:
        cnt += 1
print (cnt, 'vocab has no embedding', vocab_size)

6517 vocab has no embedding 19801


In [12]:
from keras.utils import to_categorical
data = np.array(data).reshape(17500, 35)

In [13]:
def data_generator(data):
    
    idx = np.arange(len(data))
    while True:
        np.random.shuffle(idx)
        batches = [idx[range(batch_size*i, min(len(data), batch_size*(i+1)))] for i in range(len(data)//batch_size+1)] 
        
        for i in batches:
            yield data[i], to_categorical(data[i], num_classes=vocab_size+1)

In [14]:
from keras.layers import Input, LSTM, RepeatVector, Embedding, Dense
from keras.models import Model

In [21]:
timesteps = 35
input_dim = 1
latent_dim = 128

embedding_layer = Embedding(len(t.word_index) + 1,
                            100,
                            weights=[embedding_matrix],
                            input_length=35,
                            trainable=False)

inputs = Input(shape=(timesteps,))
emb = embedding_layer(inputs)
encoded = LSTM(latent_dim)(emb)

decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(100, return_sequences=True)(decoded)
decoded = Dense(vocab_size+1, activation='softmax')(decoded)

autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)

In [22]:
autoencoder .summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 35)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 35, 100)           1980200   
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               117248    
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 35, 128)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 35, 100)           91600     
_________________________________________________________________
dense_1 (Dense)              (None, 35, 19802)         2000002   
Total params: 4,189,050
Trainable params: 2,208,850
Non-trainable params: 1,980,200
__________________________________________________________

In [23]:
from keras.optimizers import Adam
opt = Adam(lr=0.01, amsgrad=True, decay=0.005)
autoencoder.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [24]:
split = int(len(data)*0.95)
train, val = data[:split], data[split:]

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
batch_size = 32
history = autoencoder.fit_generator(data_generator(data), 
                                    steps_per_epoch=int((len(data)+batch_size-1)/batch_size), 
                                    validation_data=data_generator(val), 
                                    validation_steps=int((len(val)+batch_size-1)/batch_size),
                                    epochs=1000, callbacks=[EarlyStopping(monitor='val_loss', patience=15), 
                                                           ModelCheckpoint("autoencoder.hdf5", monitor='val_loss', save_best_only=True)])

Epoch 1/500
547/547 [==============================] - 104s 191ms/step - loss: 7.1732 - acc: 0.0635 - val_loss: 6.9110 - val_acc: 0.0729
Epoch 2/500
547/547 [==============================] - 103s 188ms/step - loss: 6.7896 - acc: 0.0782 - val_loss: 6.6184 - val_acc: 0.0821
Epoch 3/500
547/547 [==============================] - 103s 188ms/step - loss: 6.5906 - acc: 0.0862 - val_loss: 6.4658 - val_acc: 0.0900
Epoch 4/500
547/547 [==============================] - 103s 188ms/step - loss: 6.4533 - acc: 0.0935 - val_loss: 6.3419 - val_acc: 0.0963
Epoch 5/500
547/547 [==============================] - 103s 188ms/step - loss: 6.3306 - acc: 0.0997 - val_loss: 6.2279 - val_acc: 0.1029
Epoch 6/500
547/547 [==============================] - 103s 188ms/step - loss: 6.2091 - acc: 0.1051 - val_loss: 6.1120 - val_acc: 0.1081
Epoch 7/500
547/547 [==============================] - 103s 188ms/step - loss: 6.0935 - acc: 0.1102 - val_loss: 5.9853 - val_acc: 0.1129
Epoch 8/500
547/547 [====================

547/547 [==============================] - 104s 189ms/step - loss: 4.8891 - acc: 0.1855 - val_loss: 4.8276 - val_acc: 0.1891
Epoch 61/500
547/547 [==============================] - 103s 188ms/step - loss: 4.8750 - acc: 0.1865 - val_loss: 4.8080 - val_acc: 0.1960
Epoch 62/500
547/547 [==============================] - 103s 188ms/step - loss: 4.8704 - acc: 0.1868 - val_loss: 4.8067 - val_acc: 0.1925
Epoch 63/500
547/547 [==============================] - 103s 189ms/step - loss: 4.8676 - acc: 0.1875 - val_loss: 4.8211 - val_acc: 0.1912
Epoch 64/500
547/547 [==============================] - 103s 189ms/step - loss: 4.8653 - acc: 0.1879 - val_loss: 4.7983 - val_acc: 0.1960
Epoch 65/500
547/547 [==============================] - 103s 189ms/step - loss: 4.8638 - acc: 0.1884 - val_loss: 4.8121 - val_acc: 0.1904
Epoch 66/500
547/547 [==============================] - 103s 189ms/step - loss: 4.8637 - acc: 0.1895 - val_loss: 4.8102 - val_acc: 0.1925
Epoch 67/500
547/547 [=========================

547/547 [==============================] - 104s 191ms/step - loss: 4.7173 - acc: 0.2081 - val_loss: 4.6493 - val_acc: 0.2171
Epoch 120/500
547/547 [==============================] - 104s 191ms/step - loss: 4.7159 - acc: 0.2084 - val_loss: 4.6758 - val_acc: 0.2097
Epoch 121/500
547/547 [==============================] - 104s 191ms/step - loss: 4.7140 - acc: 0.2087 - val_loss: 4.6458 - val_acc: 0.2161
Epoch 122/500
547/547 [==============================] - 104s 191ms/step - loss: 4.7115 - acc: 0.2089 - val_loss: 4.6533 - val_acc: 0.2125
Epoch 123/500
547/547 [==============================] - 104s 191ms/step - loss: 4.7097 - acc: 0.2096 - val_loss: 4.6415 - val_acc: 0.2149
Epoch 124/500
547/547 [==============================] - 104s 191ms/step - loss: 4.7085 - acc: 0.2094 - val_loss: 4.6410 - val_acc: 0.2173
Epoch 125/500
547/547 [==============================] - 104s 191ms/step - loss: 4.7071 - acc: 0.2094 - val_loss: 4.6521 - val_acc: 0.2148
Epoch 126/500
547/547 [==================

In [ ]:
autoencoder.save('autoencoder.h5')
encoder.save('encoder.h5')

In [ ]:
with open('history.pkl', 'wb') as f:
    pickle.dump(history, f)